<a href="https://colab.research.google.com/github/lit2020/class-deep-learning/blob/main/%5BDL%5D201902656_%EA%B9%80%EA%B8%B0%EC%9B%85_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>